# Codigo de Clase + Test Shorts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import optuna

sns.set_theme()

In [ ]:
from dataclasses import dataclass


@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str

In [ ]:
def get_portfolio_value(cash: float, long_ops: list[Operation],
                        short_ops: list[Operation], current_price: float,
                        n_shares: int) -> float:
    val = cash

    # Add long positions value
    val += len(long_ops) * current_price * n_shares

    # Todo: Add short positions value


    return val

In [ ]:
data = pd.read_csv('aapl_5m_train.csv').dropna()
data.head()

In [ ]:
def backtest(data, trial) -> float:
    data = data.copy()

    rsi_window = trial.suggest_int('rsi_window', 5, 50)
    rsi_lower = trial.suggest_int('rsi_lower', 5, 35)
    rsi_upper = trial.suggest_int('rsi_upper', 65, 95)
    stop_loss = trial.suggest_float('stop_loss', 0.01, 0.015)
    take_profit = trial.suggest_float('take_profit', 0.01, 0.015)
    n_shares = trial.suggest_int('n_shares', 50, 500)

    rsi_indidator = ta.momentum.RSIIndicator(data.Close, window=rsi_window)
    data['rsi'] = rsi_indidator.rsi()

    historic = data.dropna()
    historic['buy_signal'] = historic['rsi']< rsi_lower  # se puede optimizar
    historic['sell_signal'] = historic['rsi'] > rsi_upper  # se puede optimizar

    COM = 0.125/100
    SL = stop_loss
    TP = take_profit
    BORROW_RATE = 0.25 / 100

    cash = 1_000_000


    active_long_positions: list[Operation] = []

    portfolio_value = [cash]

    for i, row in historic.iterrows():

        # Close positions
        for position in active_long_positions[:]:  # Iterate over a copy of the list
            if row.Close > position.take_profit or row.Close < position.stop_loss:
                # Close the position
                cash += row.Close * position.n_shares * (1 - COM)
                # Remove the position from active positions
                active_long_positions.remove(position)
                continue

        # Buy
        # Check signal
        if not row.buy_signal:
            portfolio_value.append(get_portfolio_value(
                cash, active_long_positions, [], row.Close, n_shares
            ))
            continue

        # Do we have enough cash?
        if cash < row.Close * n_shares * (1 + COM):
            portfolio_value.append(get_portfolio_value(
                cash, active_long_positions, [], row.Close, n_shares
            ))
            continue

        # Discount the cost
        cash -= row.Close * n_shares * (1 + COM)
        # Save the operation as active position
        active_long_positions.append(Operation(
            time=row.Datetime,
            price=row.Close,
            stop_loss=row.Close * (1 - SL),
            take_profit=row.Close * (1 + TP),
            n_shares=n_shares,
            type='LONG'
        ))

        # This only works for long positions
        portfolio_value.append(get_portfolio_value(
            cash, active_long_positions, [], row.Close, n_shares
        ))

    cash += row.Close * len(active_long_positions) * n_shares * (1 - COM)
    active_long_positions = []

    return(cash / 1_000_000) - 1

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: backtest(data, trial), n_trials=50)

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
rsi_indicator = ta.momentum.RSIIndicator(
    data.Close, window=33)  # la ventana se puede optimizar
data['rsi'] = rsi_indicator.rsi()

In [ ]:
historic = data.dropna()
historic['buy_signal'] = historic['rsi']< 25  # se puede optimizar
historic['sell_signal'] = historic['rsi'] > 79  # se puede optimizar

# Backtesting Clase

In [ ]:
COM = 0.125/100
SL = 0.125
TP = 0.138
BORROW_RATE = 0.25 / 100

cash = 1_000_000
n_shares = 385

In [ ]:
active_long_positions = []

portfolio_value = [cash]

for i, row in historic.iterrows():
    #This only works for long positions
    portfolio_value.append(get_portfolio_value(
        cash, active_long_positions, [], row.Close, n_shares
    ))

    # Close positions
    for position in active_long_positions[:]:  # Iterate over a copy of the list
        if row.Close > position.take_profit or row.Close < position.stop_loss:
            # Close the position
            cash += row.Close * position.n_shares * (1 - COM)
            # Remove the position from active positions
            active_long_positions.remove(position)
            continue



    # Buy
    # Check signal
    if not row.buy_signal:
        portfolio_value.append(get_portfolio_value(
        cash, active_long_positions, [], row.Close, n_shares
    ))
        continue

    # Do we have enough cash?
    if cash < row.Close * n_shares * (1 + COM):
        portfolio_value.append(get_portfolio_value(
        cash, active_long_positions, [], row.Close, n_shares
    ))
        continue

    # Discount the cost
    cash -= row.Close * n_shares * (1 + COM)
    # Save the operation as active position
    active_long_positions.append(Operation(
            time=row.Datetime,
            price=row.Close,
            stop_loss=row.Close * (1 - SL),
            take_profit=row.Close * (1 + TP),
            n_shares=n_shares,
            type='LONG'
        ))

cash += row.Close * len(active_long_positions) * n_shares * (1 - COM)
active_long_positions = []

In [ ]:
cash

In [ ]:
cash, len(active_long_positions)

In [ ]:
plt.plot(portfolio_value)